### TF - motif bias matrix

1. downloading gene - motif name mapping in here: https://resources.aertslab.org/cistarget/motif_collections/v10nr_clust_public/snapshots/motifs-v10-nr.hgnc-m0.00001-o0.0.tbl
2. then load the motif -> gene enrichment mapping in here: https://resources.aertslab.org/cistarget/databases/homo_sapiens/hg38/refseq_r80/mc_v10_clust/gene_based/the hg38_10kbp_up_10kbp_down_full_tx_v10_clust.genes_vs_motifs.scores.feather
3. then we replace the motif name with gene names
4. then we take the max enrichment if same gene-motif
5. then renormalize this per gene and save it as our new attention bias

----

### PPI bias matrix

1. we get the human PPI matrix from stringdb here: https://string-db.org/cgi/download?sessionId=bwVBywlOX6i8&species_text=Homo+sapiens
2. we get the same one for other species (e.g. mus musculus): https://stringdb-downloads.org/download/protein.links.v12.0/10090.protein.links.v12.0.txt.gz
3. we open them and use ensembl's biomart to map ENSP to ENSG. we merge all similar pairs
4. we map it to a grn with associated scores.
5. we use them as our new PPI bias matrix

In [1]:
import pandas as pd
from scdataloader.utils import load_genes
from scdataloader.utils import getBiomartTable
import numpy as np
from tqdm import tqdm
import torch
from scipy import sparse

%load_ext autoreload
%autoreload 2


/pasteur/appa/homes/jkalfon/scPRINT/.venv/lib/python3.11/site-packages/lamindb/core/storage/_anndata_accessor.py:12: FutureWarning: `__version__` is deprecated, use `importlib.metadata.version('anndata')` instead.
  from anndata import __version__ as anndata_version
/pasteur/appa/homes/jkalfon/scPRINT/.venv/lib/python3.11/site-packages/lamindb/core/storage/_zarr.py:6: FutureWarning: `__version__` is deprecated, use `importlib.metadata.version('anndata')` instead.
  from anndata import __version__ as anndata_version


→ connected lamindb: jkobject/scprint2


In [2]:
genedf = load_genes()

In [7]:
LOC = "../../data/main/" #"./data/main/"

In [4]:
da = pd.read_feather(LOC+'main_scenic+_database.feather')
da = da.set_index('motifs')


tbl = pd.read_csv(LOC+'motifs-v10-nr.hgnc-m0.00001-o0.0.tbl'
, sep='\t', header=None, names=['motif_id','motif_name',
'motif_description','source_name','source_version','gene_name','motif_similarity_qvalue','similar_motif_id','similar_motif_description','orthologous_identity','orthologous_gene_name','orthologous_species','description'])
tbl = tbl.set_index('motif_id')
tbl = tbl[tbl.gene_name.isin(genedf.symbol)]


FileNotFoundError: [Errno 2] No such file or directory: './data/main/main_scenic+_database.feather'

In [ ]:
rn = {}
toadd = {}
for i in tqdm.tqdm(da.index):
    try:
        res = tbl.loc[i, 'gene_name']
    except KeyError:
        da = da.drop(i)
    if type(res) != str:
        for v in res.values[1:]:
            toadd.update({v: da.loc[i].values})
        res = res.values[0]
    rn.update({i: res})

In [ ]:
da = da.rename(index=rn)
da = pd.concat([da, pd.DataFrame(toadd, index=da.columns).T])
da = da.loc[~da.index.duplicated(keep='first')]


In [ ]:
da.to_parquet(LOC+'main_scenic+.parquet')

In [8]:
da = pd.read_parquet(LOC+'main_scenic+.parquet')

In [ ]:
biomart = getBiomartTable(attributes=['ensembl_gene_id', "hgnc_symbol"], bypass_attributes=True)

downloading gene names from biomart
['ensembl_gene_id', 'hgnc_symbol']


In [28]:
mp = {}
for v, k in biomart.iloc[:,:2].values:
    if v is np.nan or k is np.nan:
        continue
    if k in mp:
        if v not in mp[k]:
            mp[k]+=[v]
    else:
        mp[k] = [v]


In [14]:
MAX=5
MIN=0

In [15]:
da = MIN + (da - da.min(1).values[:, None]) * (MAX - MIN) / (da.max(1) - da.min(1)).values[:, None]


In [29]:
new_index = []
new_data = []

# Remap indices
for idx in da.index:
    if idx in mp:
        for mapped_idx in mp[idx]:
            new_index.append(mapped_idx)
            new_data.append(da.loc[idx].values)
    else:
        new_index.append(idx)
        new_data.append(da.loc[idx].values)

# Remap columns
new_columns_set = set()
remapped_data = []

for row in tqdm(new_data):
    new_row = []
    for col, value in zip(da.columns, row):
        if col in mp:
            for mapped_col in mp[col]:
                new_row.append(value)
                new_columns_set.add(mapped_col)
        else:
            new_row.append(value)
            new_columns_set.add(col)
    remapped_data.append(new_row)

new_columns = list(new_columns_set)

new_da = pd.DataFrame(remapped_data, index=new_index, columns=new_columns)


100%|██████████| 1692/1692 [00:11<00:00, 144.60it/s]


ValueError: 30498 columns passed, passed data had 30500 columns

In [30]:
new_columns

['ENSG00000180016',
 'ENSG00000136040',
 'LOC101926933',
 'ENSG00000007350',
 'ENSG00000163539',
 'ENSG00000276221',
 'ENSG00000204478',
 'ENSG00000231676',
 'ENSG00000163964',
 'ENSG00000275136',
 'ENSG00000119682',
 'ENSG00000116685',
 'LOC105377143',
 'FAM231B',
 'ENSG00000284979',
 'ENSG00000108797',
 'HSPC324',
 'ENSG00000154001',
 'LOC100506207',
 'LOC100506368',
 'ENSG00000138136',
 'ENSG00000200367',
 'C7orf55',
 'ENSG00000196544',
 'ENSG00000236751',
 'LOC100505918',
 'ENSG00000213401',
 'ENSG00000090097',
 'ENSG00000284013',
 'ENSG00000175416',
 'ENSG00000166136',
 'ENSG00000064042',
 'ENSG00000223638',
 'SPACA6P-AS',
 'ENSG00000147378',
 'FAM159B',
 'HCG16',
 'ENSG00000008988',
 'ENSG00000211590',
 'ENSG00000146535',
 'ENSG00000221649',
 'ENSG00000052126',
 'ENSG00000187733',
 'ENSG00000134248',
 'ENSG00000108829',
 'ENSG00000166793',
 'ENSG00000229027',
 'ENSG00000184434',
 'ENSG00000073792',
 'ENSG00000291865',
 'ENSG00000144218',
 'ENSG00000118369',
 'ENSG00000281934',
 '

In [ ]:
new_da

,CADM2-AS2,PHGR1,LOC101926933,MIR3119-1,TLN2,VSTM2A,ZDHHC24,PQBP1,LOC105377143,FAM231B,ADCY3,RPS15,PBDC1,HSPC324,APOBEC4,UTP18,LOC100506207,LOC100506368,SNX29P2,C7orf55,MIR5007,IGF2BP3,LOC100505918,SPACA6P-AS,IKZF4,FAM159B,HCG16,MIR4769,CD70,MIOX,PRNT,NEO1,CD55,DAND5,CCDC158,MVB12A,SH2D1A,DEFB112,RLIM,TP53BP1,EPB41L2,UCP2,ZNF546,DDC-AS1,MAP4K4,LINC00572,PPARD,CHKB-CPT1B,LOC101929372,IGFL3,BCOR,LINC01257,LOC101928551,ATXN1L,FLJ38576,TTC7A,FAM159A,MIR6770-3,BMP8A,L3MBTL4-AS1,SNORD3A,LOC102723885,MANBA,LOC152225,ARHGAP35,ELP4,DAB2IP,LOC100505920,LOC101928307,GOLGA6L17P,KNG1,DZANK1,DLX5,SMS,YBEY,OR51G1,MIR638,COL3A1,DSTN,MIR4259,MIR548AB,NLGN4X,SHISA9,USP3,RAP2B,MIR6792,ADAMTS18,TMTC2,LINC01048,S100A1,ROGDI,MIR466,TAF4,ATG4C,HRH3,FGFBP2,ALG10B,FRAT1,AGTPBP1,LRP5L,...,LOC100507412,PLIN4,ACKR2,PCDHA9,MIR4477B,LINC01184,SLC16A4,MBD4,ERICH3,MIR181A2,MMP25-AS1,CELA3B,MAFF,GLIS2,KCTD14,MIR6828,GTF3C6,LINC00308,LOC105372280,LOC105373100,SULT1E1,SRM,CHAD,LINC01204,FUT4,SRD5A1,TAS2R5,TMEM132E,TTC9,ATG13,PRDM4,SNORD112,C16orf59,ITGAD,LOC283683,SLC6A17,NCK2,ATL3,MIR1244-4,WFDC10B,NFASC,SNORA59B,UEVLD,SOX8,MEIOB,WIPF2,KIF3B,GSTA1,ZNF525,HOXA5,MYO1D,SNORD109B,TMEM189-UBE2V1,ARRDC1,MUC7,RPP14,TNS3,JCHAIN,LINC01508,SAMD1,TTTY23B,OR5M1,ZC3HC1,SNORD31,MUM1L1,UQCRBP1,RNF175,LRRTM4,SLC15A5,MIR3668,SRC,LOC101928812,C6orf141,ATAD1,GEMIN6,LINC01467,SNORD86,CDK2,MEF2A,LOC101929161,WDR43,FLJ31104,NFYC,CLTC,KPNA6,LOC101928435,ST3GAL5-AS1,ZNF527,CMSS1,H1FX,FAP,C2CD4D,MDGA1,TCEB1,CLASP2,PTGS1,LOC105373876,MIR4713,NRIP1,CES1
RBPJ,0.060400,0.060800,0.197400,0.128600,0.082600,0.149600,0.095200,0.150200,0.125000,0.119600,0.123600,0.147600,0.181400,0.126000,0.090200,0.083600,0.177000,0.156400,0.086200,0.097400,0.100200,0.105400,0.093000,0.087600,0.130600,0.135800,0.076400,0.150000,0.230000,0.106400,0.125400,0.141200,0.079000,0.069000,0.154000,0.117000,0.121800,0.162000,0.161800,0.143400,0.109000,0.164800,0.094600,0.081600,0.146800,0.116800,0.063400,0.102400,0.131800,0.145000,0.202000,0.133000,0.153200,0.089200,0.116800,0.094800,0.191400,0.140200,0.082000,0.076000,0.175000,0.172800,0.128400,0.089000,0.136200,0.130400,0.110600,0.151200,0.112400,0.146600,0.079400,0.138400,0.097200,0.086600,0.111200,0.081800,0.074200,0.067000,0.070000,0.099800,0.187200,0.142800,0.078800,0.116400,0.139400,0.086400,0.136400,0.067200,0.178400,0.187400,0.254000,0.126400,0.122400,0.107600,0.115000,0.155000,0.066200,0.122400,0.108400,0.143600,...,0.172400,0.128600,0.124000,0.091200,0.065200,0.116600,0.187400,0.073800,3.040000,0.074000,0.166400,0.071200,0.069800,0.124800,0.148600,0.124400,0.119000,0.108400,0.100600,0.085000,0.137200,0.131400,0.100800,0.131000,0.068800,0.152600,0.236000,0.222000,0.117000,0.070400,0.068200,0.058600,0.122800,0.093800,0.126600,0.127400,0.136000,0.162400,0.188000,0.222000,0.129800,0.126200,0.070400,0.110600,0.129000,0.143000,0.076000,0.091600,0.111400,0.262000,0.115000,0.085200,0.121200,0.131200,0.085600,0.076600,0.078200,0.143400,0.176200,0.198200,0.078400,0.111400,0.101800,0.131000,0.166400,0.154200,0.083600,0.194000,0.145200,0.154800,0.127200,0.116000,0.125400,0.103200,0.114400,0.190800,0.978000,0.146800,0.128400,0.240000,0.142200,0.092000,0.090000,0.111400,0.166000,0.061800,0.256000,0.122000,0.148600,0.119800,0.086400,0.141000,0.226000,0.177200,0.135400,0.067000,0.081400,0.151200,0.131000,0.131400
FOXC2,0.387407,0.362222,0.851852,0.650370,0.445185,0.371852,0.304444,0.429630,0.537778,0.396296,0.337778,0.371111,0.785185,0.400741,0.478519,0.400000,0.403704,0.619259,0.242222,0.325185,0.417778,0.370370,0.814815,0.560741,0.339259,0.628889,0.419259,0.434815,0.390370,0.208889,0.358519,0.268889,0.540000,0.962963,0.529630,0.837037,0.408889,0.417037,0.415556,0.340741,0.495556,0.443704,0.532593,0.599259,0.512593,0.455556,1.429630,0.038074,1.718518,0.455556,0.384444,0.442963,0.388889,0.544444,0.393333,0.242222,0.662222,0.440741,0.911111,0.343704,0.718518,0.282222,0.520741,0.356296,0.382222,0.393333,0.601481,0.301481,0.405926,0.377778,0.445185,0.711852,0.474074,0.658519,

In [18]:
# Ensure new_da is a square matrix with the union of both columns and indices
all_labels = sorted(set(new_da.index).union(set(new_da.columns)))
new_da = new_da.reindex(index=all_labels, columns=all_labels, fill_value=0)
new_da


,A1BG,A1BG-AS1,A1CF,A2M,A2M-AS1,A2ML1,A2MP1,A3GALT2,A4GALT,A4GNT,AA06,AAAS,AACS,AACSP1,AADAC,AADACL2,AADACL2-AS1,AADACL3,AADACL4,AADACP1,AADAT,AAED1,AAGAB,AAK1,AAMDC,AAMP,AANAT,AAR2,AARD,AARS,AARS2,AARSD1,AASDH,AASDHPPT,AASS,AATBC,AATF,AATK,AATK-AS1,ABALON,ABAT,ABCA1,ABCA10,ABCA11P,ABCA12,ABCA13,ABCA17P,ABCA2,ABCA3,ABCA4,ABCA5,ABCA6,ABCA7,ABCA8,ABCA9,ABCA9-AS1,ABCB1,ABCB10,ABCB11,ABCB4,ABCB5,ABCB6,ABCB7,ABCB8,ABCB9,ABCC1,ABCC10,ABCC11,ABCC12,ABCC13,ABCC2,ABCC3,ABCC4,ABCC5,ABCC5-AS1,ABCC6,ABCC6P1,ABCC6P2,ABCC8,ABCC9,ABCD1,ABCD2,ABCD3,ABCD4,ABCE1,ABCF1,ABCF2,ABCF3,ABCG1,ABCG2,ABCG4,ABCG5,ABCG8,ABHD1,ABHD10,ABHD11,ABHD11-AS1,ABHD12,ABHD12B,ABHD13,...,ZNF84,ZNF841,ZNF843,ZNF844,ZNF845,ZNF846,ZNF85,ZNF850,ZNF852,ZNF853,ZNF860,ZNF862,ZNF865,ZNF875,ZNF876P,ZNF878,ZNF879,ZNF880,ZNF883,ZNF888,ZNF890P,ZNF891,ZNF90,ZNF91,ZNF92,ZNF93,ZNF98,ZNF99,ZNFX1,ZNHIT1,ZNHIT2,ZNHIT3,ZNHIT6,ZNRD1,ZNRD1ASP,ZNRF1,ZNRF2,ZNRF2P1,ZNRF2P2,ZNRF3,ZNRF3-AS1,ZNRF4,ZP1,ZP2,ZP3,ZP4,ZPBP,ZPBP2,ZPLD1,ZPR1,ZRANB1,ZRANB2,ZRANB2-AS1,ZRANB2-AS2,ZRANB3,ZRSR2,ZSCAN1,ZSCAN10,ZSCAN12,ZSCAN12P1,ZSCAN16,ZSCAN16-AS1,ZSCAN18,ZSCAN2,ZSCAN20,ZSCAN21,ZSCAN22,ZSCAN23,ZSCAN25,ZSCAN26,ZSCAN29,ZSCAN30,ZSCAN31,ZSCAN32,ZSCAN4,ZSCAN5A,ZSCAN5B,ZSCAN5C,ZSCAN9,ZSWIM1,ZSWIM2,ZSWIM3,ZSWIM4,ZSWIM5,ZSWIM6,ZSWIM7,ZSWIM8,ZSWIM8-AS1,ZUFSP,ZW10,ZWILCH,ZWINT,ZXDA,ZXDB,ZXDC,ZYG11A,ZYG11B,ZYX,ZZEF1,ZZZ3
A1BG,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.0000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000
A1BG-AS1,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.0000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0

In [19]:
new_da = new_da.loc[new_da.index.str.contains('ENSG'),new_da.index.str.contains('ENSG')]

In [22]:
new_da

""


In [ ]:
new_da = new_da.T + new_da

## protein

In [17]:
string = pd.read_csv('../data/main/9606.protein.links.v12.0.txt.gz', sep=' ')

In [20]:
del string

In [18]:
string.protein1 = string.protein1.str.split('.').str[1]
string.protein2 = string.protein2.str.split('.').str[1]

In [10]:
rel = {}
for a, b in string.iloc[:,:2].values:
    a, b = rn.get(a, ""), rn.get(b, "")
    if a and b:
        if a in rel:
            rel[a].add(b)
        else:
            rel[a] = set([b])  
        if b in rel:
            rel[b].add(a)
        else:
            rel[b] = set([a])
len(rel)

19193

In [26]:
res = pd.DataFrame(data=np.zeros((len(rel), len(rel))), columns=list(rel.keys()), index=list(rel.keys()))

In [12]:
res.drop(columns=[''], inplace=True)
res.drop(index=[''], inplace=True)

In [25]:
for i,j in tqdm.tqdm(rel.items()):
    res.loc[i,list(j)] = 1


19193it [00:20, 917.76it/s] 


In [ ]:
res.mean().mean()

0.03608647264434039

In [ ]:
res.to_parquet('../data/main/stringdb_bias.parquet')

In [21]:
res = pd.read_parquet("../data/main/stringdb_bias.parquet")

In [30]:
res

,ENSG00000075292,ENSG00000172531,ENSG00000180745,ENSG00000156886,ENSG00000185069,ENSG00000125817,ENSG00000177459,ENSG00000074201,ENSG00000109390,ENSG00000165066,...,ENSG00000152592,ENSG00000164778,ENSG00000130957,ENSG00000165084,ENSG00000114487,ENSG00000178093,ENSG00000203950,ENSG00000126953,ENSG00000185985,ENSG00000180210
ENSG00000075292,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
ENSG00000172531,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
ENSG00000180745,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
ENSG00000156886,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
ENSG00000185069,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ENSG00000178093,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
ENSG00000203950,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
ENSG00000126953,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
ENSG00000185985,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
res + res.T

In [29]:
res.sum(1)

ENSG00000075292    1221.0
ENSG00000172531    3264.0
ENSG00000180745     466.0
ENSG00000156886     287.0
ENSG00000185069     671.0
                    ...  
ENSG00000178093    1031.0
ENSG00000203950       0.0
ENSG00000126953     852.0
ENSG00000185985     719.0
ENSG00000180210    1276.0
Length: 19193, dtype: float64

In [66]:
# # Align the indices and columns of res with da, and vice versa, filling missing values with 0
all_indices = sorted(set(new_da.index).union(set(res.index)))
# 
da_aligned = new_da.reindex(index=all_indices, columns=all_indices, fill_value=0)
res_aligned = res.reindex(index=all_indices, columns=all_indices, fill_value=0)
# 
# # Add res to da
da = da_aligned + res_aligned

In [67]:
da.shape

(28206, 28206)

In [68]:
del new_da
del res

In [16]:
genes = torch.load('../data/temp/vbd8bavn/epoch=17-step=90000.ckpt')['hyper_parameters']['genes']

In [ ]:
biomart = getBiomartTable(attributes=['mmusculus_homolog_ensembl_gene', 'ensembl_gene_id'], bypass_attributes=True)
biomart

downloading gene names from biomart
['mmusculus_homolog_ensembl_gene', 'ensembl_gene_id']


,mmusculus_homolog_ensembl_gene,ensembl_gene_id
0,NaN,ENSG00000210049
1,NaN,ENSG00000211459
2,NaN,ENSG00000210077
3,NaN,ENSG00000210082
4,NaN,ENSG00000209082
...,...,...
77642,NaN,ENSG00000232679
77643,ENSMUSG00000088001,ENSG00000200033
77644,NaN,ENSG00000228437
77645,NaN,ENSG00000229463


In [ ]:
biomart[~biomart.mmusculus_homolog_ensembl_gene.isna()]

,mmusculus_homolog_ensembl_gene,ensembl_gene_id
5,ENSMUSG00000064341,ENSG00000198888
9,ENSMUSG00000064345,ENSG00000198763
15,ENSMUSG00000064351,ENSG00000198804
18,ENSMUSG00000064354,ENSG00000198712
20,ENSMUSG00000064356,ENSG00000228253
...,...,...
77627,ENSMUSG00000032913,ENSG00000198799
77639,ENSMUSG00000102439,ENSG00000143631
77640,ENSMUSG00000049133,ENSG00000143520
77641,ENSMUSG00000039384,ENSG00000143507


In [79]:
mp = {} 
for k,v in biomart.iloc[:,[0,2]].values:
    if v is np.nan or k is np.nan:
        continue
    if k in mp:
        if v not in mp[k]:
            mp[k]+=[v]
    else:
        mp[k] = [v]

In [81]:
# Ensure da contains exactly the same genes as in genes (dropping or filling empty with 0)
da = da.reindex(index=genes, columns=genes, fill_value=0)
# Convert da to a sparse array
da = sparse.csr_matrix(da.values)


In [ ]:
biomart_filtered = biomart[~biomart.mmusculus_homolog_ensembl_gene.isna()].iloc[:, [0, 2]]
biomart_filtered = biomart_filtered.loc[~biomart_filtered.duplicated(keep=False)]

biomart_filtered[biomart_filtered.mmusculus_homolog_ensembl_gene.isin(set(genes)) & biomart_filtered.ensembl_gene_id.isin(set(genes))]
biomart_filtered


,ensembl_gene_id,mmusculus_homolog_ensembl_gene
5,ENSG00000198888,ENSMUSG00000064341
9,ENSG00000198763,ENSMUSG00000064345
15,ENSG00000198804,ENSMUSG00000064351
18,ENSG00000198712,ENSMUSG00000064354
20,ENSG00000228253,ENSMUSG00000064356
...,...,...
76056,ENSG00000116771,ENSMUSG00000040706
76059,ENSG00000252417,ENSMUSG00002075659
76062,ENSG00000215695,ENSMUSG00000078515
76063,ENSG00000215695,ENSMUSG00000040715


In [ ]:
# Create a mapping from gene to its index in the genes list
gene_to_index = {gene: idx for idx, gene in enumerate(genes)}

# Replace values in biomart_filtered with their locations in "genes"
biomart_filtered = biomart_filtered.applymap(lambda x: gene_to_index.get(x, x))

# Filter out rows where any of the values are not in the gene_to_index mapping
biomart_filtered = biomart_filtered[biomart_filtered.applymap(lambda x: isinstance(x, int)).all(axis=1)]


/tmp/ipykernel_1168549/621159413.py:5: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  biomart_filtered = biomart_filtered.applymap(lambda x: gene_to_index.get(x, x))
/tmp/ipykernel_1168549/621159413.py:8: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  biomart_filtered = biomart_filtered[biomart_filtered.applymap(lambda x: isinstance(x, int)).all(axis=1)]


In [ ]:
da[biomart_filtered.values[:,1][:, None], biomart_filtered.values[:,1]] = da[biomart_filtered.values[:,0][:, None], biomart_filtered.values[:,0]]

/home/ml4ig1/miniconda3/envs/scprint/lib/python3.10/site-packages/scipy/sparse/_index.py:134: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  self._set_arrayXarray_sparse(i, j, x)


In [ ]:
sparse.save_npz("bias_sparse.npz", da)


In [25]:
sparse.save_npz("bias_sparse.npz", da)
